In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import histogram


In [31]:

data=pd.read_csv("./annotations.csv")
files=data["fname"]   #read file names in sequential order as in annotations.csv


x=np.zeros((1000,128))
flagg=np.zeros((1000,3))  #flag variable to store features of all data points
temp1=np.zeros((19,128))

j=0

for i in files:
    temp=np.load("./train/"+ i)
    frames=temp.shape[2]
    flagg[j,0]=frames     #no.of data frames given in each data-point
    
    #repeat the spectrogram if file width is less than 200 since we will require minimum 200 frames
    if frames<100:
        temp=np.tile(temp,300//frames+1)
        frames=temp.shape[2]
    
    #find max column vect
    sum_ver=np.sum(temp,axis=1)
    t=np.argmax(sum_ver)   
    flagg[j,1]=t
    
    
    # selects a snipet of spectrogram of width 100 as sample
    if flagg[j,1]<100:
        temp=temp[:,:,0:200]
    elif flagg[j,1]>frames-100:
        temp=temp[:,:,-200:]
    else:
        temp=temp[:,:,t-100:t+100]
    
    #avg contribution of each frequency temporally
    temp=np.sum(temp,axis=-1)/200

    x[j]=temp#training data
    
    flagg[j,2]=np.sum(x[j])  #to find outliers in training data
    j=j+1

print(x.shape)

#Averaging all train sample of shape (1,2432) to values less than 1 for easier calculation of softmax
x_soft=x/(np.sum(x,axis=1).reshape(1000,1)+0.00000000000000000000001)
print(x_soft.shape)


(1000, 128)


(1000, 128)


In [33]:
from sklearn import preprocessing

encode=preprocessing.LabelEncoder()
y1 = encode.fit_transform(data["label"]) #every classifier is encodded with a number
y= (pd.get_dummies(y1).values) #number  is converter to one-hot vector


t2=y1[0:1000:100]

dict_class={}   #dictionary of one hot vector with respective class

for i in range(0,10):
    j=int(np.argwhere(t2==i))
    dict_class[i]=data["label"][100*j]
print(dict_class)


{0: 'Bark', 1: 'Crying_and_sobbing', 2: 'Doorbell', 3: 'Knock', 4: 'Meow', 5: 'Microwave_oven', 6: 'Shatter', 7: 'Siren', 8: 'Vehicle_horn_and_car_horn_and_honking', 9: 'Walk_and_footsteps'}


In [34]:
p=100
q=100-p
x_train=x_soft[0:p]
y_train=y[0:p]
print(x_train.shape)
for i in range(1,10):
    x_train=np.concatenate((x_train,x_soft[i*100:i*100+p]))
    y_train=np.concatenate((y_train,y[i*100:i*100+p]))
    

print(x_train.shape)
print(y_train.shape)


(100, 128)
(1000, 128)
(1000, 10)


In [35]:
w1=np.ones((128,10))
i=0
while i<25000:
    
    Y=np.matmul(x_train,w1)
    div=np.sum(np.exp(Y),axis=1).reshape(10*p,1)
    y_hat=np.divide(np.exp(Y),div)
    foo=-np.multiply(y_train,np.log(y_hat))
    error=np.sum(foo)/(10*p)
    print(error)
    print(i)

    dEdY=(y_hat-y_train)
    dEdw1=np.matmul(x_train.T,dEdY)
    
    w1=w1-2.3*dEdw1
    
    i=i+1

2.302585092994046
0
2.191530627064993
1
2.101575880453337
2
2.0275844135780225
3
1.9651929902430383
4
1.911611420922679
5
1.864856396091322
6
1.8235621601041567
7
1.786750178779569
8
1.7536391013479895
9
1.7236648190236896
10
1.6963378025483087
11
1.6712960578287233
12
1.6482212085481274
13
1.6268645116532143
14
1.6070139223276967
15
1.5884909179040503
16
1.571150414149876
17
1.5548607216859027
18
1.539519345005968
19
1.5250274230047922
20
1.5113100276950295
21
1.4982920061758946
22
1.485916723487731
23
1.4741263993518527
24
1.462876931315716
25
1.452123353401113
26
1.4418304752338311
27
1.4319628322083495
28
1.4224918127534734
29
1.413389048588427
30
1.404630943570099
31
1.3961944838710942
32
1.3880599937072697
33
1.380208551153493
34
1.372623601886478
35
1.3652893960490338
36
1.358191898744496
37
1.351317855762018
38
1.3446552869536876
39
1.3381929299983726
40
1.3319204946138812
41
1.3258283296223061
42
1.3199075444994133
43
1.3141498069219228
44
1.3085473933063922
45
1.3030930623559

In [37]:

data_test=pd.read_csv("./Audio_Classification-MLSP-test/files.csv")
files_test=data_test["files.csv"]   #read file names in sequential order as in annotations.csv

sample=201
x_test=np.zeros((201,128))
flagg2=np.zeros((201,3))  #flag variable to store features of all data points


j=0

for i in files_test:
    temp=np.load("./Audio_Classification-MLSP-test/"+ i)
    frames=temp.shape[2]
    flagg2[j,0]=frames     #no.of data frames given in each data-point
    
    #repeat the spectrogram if file width is less than 200 since we will require minimum 200 frames
    if frames<100:
        temp=np.tile(temp,300//frames+1)
        frames=temp.shape[2]
    2
    #find max column vect
    sum_ver=np.sum(temp,axis=1)
    t=np.argmax(sum_ver)   
    flagg2[j,1]=t
    
    
    # selects a snipet of spectrogram of width 100 as sample
    if flagg2[j,1]<100:
        temp=temp[:,:,0:200]
    elif flagg2[j,1]>frames-100:
        temp=temp[:,:,-200:]
    else:
        temp=temp[:,:,t-100:t+100]
    
    #avg contribution of each frequency temporally
    temp=np.sum(temp,axis=-1)/200

    x_test[j]=temp#training data
    
    flagg2[j,2]=np.sum(x[j])  #to find outliers in training data
    j=j+1

print(x_test.shape)

#Averaging all train sample of shape (1,2432) to values less than 1 for easier calculation of softmax
x_test=x_test/(np.sum(x_test,axis=1).reshape(201,1)+0.00000000000000000000001)
print(x_test.shape)



(201, 128)
(201, 128)


In [51]:
Y_test=np.matmul(x_test,w1)
y_hat_test=np.divide(np.exp(Y_test),np.sum(np.exp(Y_test),axis=1).reshape(201,1))

data_test["prediction"]=np.ones(201).T

for i in range(0,201):
    data_test["prediction"][i]=dict_class[np.argmax(y_hat_test[i])]


data_test.to_csv("predictions_nn.csv")

/tmp/ipykernel_17092/232946291.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test["prediction"][i]=dict_class[np.argmax(y_hat_test[i])]


In [42]:
print(y_hat_test[0])
print(np.argmax(y_hat_test[0]))
print(dict_class[4])

[3.77889090e-02 2.41411685e-04 1.19709809e-04 3.66372094e-02
 7.16229017e-01 2.76032774e-02 4.02991966e-10 9.93682948e-05
 5.47706402e-04 1.80733390e-01]
4
Meow


In [53]:
filew=open("weights_nn.txt","w")
str=repr(w1)
filew.write("NN"+"\n"+str+"\n")
filew.close()